In [1]:
# Import required libraries and user sensible data
import pandas as pd
import sweetviz as sv
import warnings
from sqlalchemy import create_engine
from sql.postgres_connection import (dbname, password, host, port, database)
from sql.sql_queries import (sql_query)
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder

# Suppress FutureWarning
warnings.simplefilter(action='ignore', category=FutureWarning)

# Database connection function
def read_data_from_postgres(query):
    try:
        engine = create_engine(f'postgresql://{dbname}:{password}@{host}:{port}/{database}')
        df = pd.read_sql_query(query, engine)
        
    except Exception as e:
        print("An error occurred:", e)

    return df

df = read_data_from_postgres(sql_query)

df_original = df.copy()

df.describe()

,it_co_orgao_superior,it_co_orgao_vinculado,it_co_unidade_gestora
count,35955.000000,35955.000000,35955.000000
mean,48878.421610,52958.121902,532517.876790
std,36824.267604,31463.068214,336193.277361
min,1000.000000,1000.000000,974.000000
25%,20101.000000,25000.000000,171109.500000
50%,20101.000000,41000.000000,463135.000000
75%,99900.000000,94000.000000,927317.500000
max,99900.000000,99900.000000,999998.000000


In [2]:
# SweetViz Setup
feature_config = sv.FeatureConfig(skip=()) # Possible parameters: skip, force_cat, force_num and force_text
my_report = sv.analyze(df, feat_cfg=feature_config)

# Export Report to HTML
my_report.show_html(filepath='sweetviz_report.html', 
                    open_browser=True, 
                    layout='vertical', 
                    scale=None)

# # Print Report to Notebook
# my_report.show_notebook(w="100%",
#                         h=None, 
#                         scale=None, 
#                         layout='vertical',
#                         filepath=None)

                                             |          | [  0%]   00:00 -> (? left)

Report sweetviz_report.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [5]:
df = df_original.copy()

# Encode categorical columns from concatenation of previous levels to allow ordering by catalogue hierarchy
df['it_co_orgao_superior_norm'] = df['it_co_orgao_superior'].copy()
df['it_co_orgao_vinculado_norm'] = df['it_co_orgao_superior_norm'].astype(str) + df['it_co_orgao_vinculado'].astype(str).str.pad(width=5, side='left', fillchar='0')
df['it_co_unidade_gestora_norm'] = df['it_co_orgao_vinculado_norm'].astype(str) + df['it_co_unidade_gestora'].astype(str).str.pad(width=6, side='left', fillchar='0')

# Encode categorical columns to numeric values
df['it_co_orgao_superior_norm'] = OrdinalEncoder().fit_transform(df[['it_co_orgao_superior_norm']])
df['it_co_orgao_vinculado_norm'] = OrdinalEncoder().fit_transform(df[['it_co_orgao_vinculado_norm']])
df['it_co_unidade_gestora_norm'] = OrdinalEncoder().fit_transform(df[['it_co_unidade_gestora_norm']])

# Check new columns
# df[['it_co_orgao_superior_norm', 'it_co_orgao_vinculado_norm', 'it_co_unidade_gestora_norm']].head(10)
df.describe()

,it_co_orgao_superior,it_co_orgao_vinculado,it_co_unidade_gestora,it_co_orgao_superior_norm,it_co_orgao_vinculado_norm,it_co_unidade_gestora_norm
count,35955.000000,35955.000000,35955.000000,35955.000000,35955.000000,35955.000000
mean,48878.421610,52958.121902,532517.876790,40.637742,74.119761,17977.000000
std,36824.267604,31463.068214,336193.277361,33.921083,66.990559,10379.458801
min,1000.000000,1000.000000,974.000000,0.000000,0.000000,0.000000
25%,20101.000000,25000.000000,171109.500000,14.000000,18.000000,8988.500000
50%,20101.000000,41000.000000,463135.000000,14.000000,31.000000,17977.000000
75%,99900.000000,94000.000000,927317.500000,88.000000,165.000000,26965.500000
max,99900.000000,99900.000000,999998.000000,88.000000,170.000000,35954.000000
